# Running apps in the dashboard

This notebook describes how to run your apps from the streamlit dashboard. Following this notebook, you should be able to access your apps and interact with them within the streamlit dashboard under the **Apps** page (see screenshot below). Make sure to check the **Setting up** section below to get your app in the list of apps on that page.

![App Runner](https://www.trulens.org/assets/images/appui/apps.png)

Clicking *New session* under any of these apps will bring up an empty transcript of the interactions between the user (you) and the app (see screenshot below). Typing a message under *Your message* on the bottom of the window, and pressing enter, will run your app with that specified message as input, produce the app output, and add both to the chat transcript under the *Records* column.

![Blank Session](https://www.trulens.org/assets/images/appui/blank_session.png)

Several other inputs are present on this page which control what about the produced transcript record to show alongside their inputs/outputs.

- Under the *App details* heading, you can specify Selectors of components of your app which then shows them in that column as the transcript is produced. These selectors are the same specifications as seen in the green labels in other parts of the Dashboard. 

- Under the *Records* heading, you can add Selectors of record parts in a similar manner. Each added selectors will then be presented alongside each input-output pair in the transcript.

Note: When specifying selectors, you skip the "Select.App" or "Select.Record" part of those selectors. Also the "RecordInput" and "RecordOutput" (not that you would need them given they are in the transcript already) are specified as "main_input" and "main_output", respectively. 

An example of a running session with several selectors is shown in the following screenshot:

![Running Session](https://www.trulens.org/assets/images/appui/running_session.png)

The session is preserved when navigating away from this page, letting you inspect the produced records in the **Evaluation** page, for example. To create a new session, you first need to end the existing one by pressing the "End session" button on top of the runner page.

## Setting up

### App loader

To be able to create a new session or "conversation", we need to be able to
reset the langchain app to its initial state. For this purpose, we require the
callable that produces a new chain that is configured for the start of the
conversation. Things like memory or other stateful aspects of the chain should
be at their initial values. Because of this, we need to construct all components
that could theoretically be stateful fully inside the required callable.

**NOTE**: We impose a limit on how big the serialization of the loader is. To
reduce its size, do not rely on globals defined outside of the function to
implement its functionality. The llama_index example in this notebook shows a
case where it may be a good idea to include a global (i.e. something downloaded
from the web). 

**WARNING**: This function needs to return a new instance of the app independent
of any others produced earlier. That is, you cannot take an existing or
pre-loaded app, clear its memory, and return it. As part of the dashboard,
multiple instances of an app need to operate at the same time without
interference in their states.

## langchain example

In [ ]:
def load_langchain_app():
    # All relevant imports must be inside this function.

    from langchain.chains import ConversationChain
    from langchain.memory import ConversationSummaryBufferMemory
    from langchain_community.llms import OpenAI

    llm = OpenAI(temperature=0.9, max_tokens=128)

    # Conversation memory.
    memory = ConversationSummaryBufferMemory(
        max_token_limit=64,
        llm=llm,
    )

    # Conversational app puts it all together.
    app = ConversationChain(llm=llm, memory=memory)

    return app


app1 = load_langchain_app()

tru_app1 = tru.Chain(
    app1, app_id="langchain_app", initial_app_loader=load_langchain_app
)

## llama_index example

In [ ]:
from llama_index.readers.web import SimpleWebPageReader

# Be careful what you include as globals to be used by the loader function as it
# will have to be serialized. We enforce a size limit which prohibits large
# objects to be included in the loader's closure.

# This object will be serialized alongside `load_llamaindex_app` below.
documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["http://paulgraham.com/worked.html"]
)


def load_llamaindex_app():
    from llama_index.core import VectorStoreIndex

    index = VectorStoreIndex.from_documents(documents)
    query_engine = index.as_query_engine()

    return query_engine


app2 = load_llamaindex_app()
tru_app2 = tru.Llama(
    app2, app_id="llamaindex_app", initial_app_loader=load_llamaindex_app
)

## basic app example

In [ ]:
from trulens.core.app import TruWrapperApp


def load_basic_app():
    def custom_application(prompt: str) -> str:
        return f"a useful response to {prompt}"

    return TruWrapperApp(custom_application)


app3 = load_basic_app()

tru_app3 = tru.Basic(
    app3, app_id="basic_app", initial_app_loader=load_basic_app
)

## custom app example

In [ ]:
from examples.expositional.end2end_apps.custom_app.custom_app import (
    CustomApp,  # our custom app
)


# Create custom app:
def load_custom_app():
    app = CustomApp()
    return app


app4 = load_custom_app()

# Create trulens wrapper:
tru_app4 = tru.Custom(
    app=app4,
    app_id="custom_app",
    # Make sure to specify using the bound method, bound to self=app.
    main_method=app4.respond_to_query,
    initial_app_loader=load_custom_app,
)

## Verification

You can get a list of apps that include the `initial_app_loader` with the following utility method.

In [ ]:
from trulens.core.schema import AppDefinition

for app_json in AppDefinition.get_loadable_apps():
    print(app_json["app_id"])